# Starting with 1 and spiralling anticlockwise in the following way, a square spiral with side length 7 is formed.

![](images/58.PNG)

# It is interesting to note that the odd squares lie along the bottom right diagonal, but what is more interesting is that 8 out of the 13 numbers lying along both diagonals are prime; that is, a ratio of 8/13 ≈ 62%.

# If one complete new layer is wrapped around the spiral above, a square spiral with side length 9 will be formed. If this process is continued, what is the side length of the square spiral for which the ratio of primes along both diagonals first falls below 10%?

_____

In [199]:
import pandas as pd
import numpy as np

In [200]:
def outer_layer(df_spiral, list_values):
    rows, columns = df_spiral.shape
    step_size = columns - 1
    
    list_1 = list_values[:step_size]
    list_2 = list_values[step_size:2*step_size]
    list_3 = list_values[2*step_size:3*step_size]
    list_4 = list_values[3*step_size:]
    
    i = -1
    for val in list_1:
        i+=1
        df_spiral.iloc[-2-i,-1] = val
        
    
    j = 0
    for val in list_2:
        j+=1
        df_spiral.iloc[-2-i,-1-j] = val        
        
    
    for val in list_3:
        i-=1
        df_spiral.iloc[-2-i,-1-j] = val
        
    for val in list_4:
        j-=1
        df_spiral.iloc[-2-i,-1-j] = val
         
    return df_spiral

In [201]:
def build_spiral(n, df):
    if n == 1:
        return pd.DataFrame([1])
    
    if df == 0:
        df = build_spiral(n-1,0)
        
    rows, cols = df.shape
    zero_array = np.zeros(cols + 2)
    df_new = pd.DataFrame(columns = range(cols+2))
    df_new.loc[0] = zero_array
    for row in df.index:
        df_new.loc[row+1] = [0]+list(df.loc[row].values)+[0]
    df_new.loc[row+2] = zero_array
    
    max_val = df.values.max()
    n_new_elements = df_new.size - df.size
    list_new_elements = list(range(max_val + 1, max_val + n_new_elements + 1))
    
    df_new = outer_layer(df_new, list_new_elements)
    
    return df_new.astype(int)

In [202]:
def is_prime(n):
    max_val = (n**0.5)//1+1
    for i in range(2, int(max_val)):
        if n%i==0:
            return False
    return True

In [203]:
df_spiral = build_spiral(1, 0)

In [ ]:
found = False

n = 1
df_spiral = build_spiral(n, 0)

while not found:
    n += 1
    df_spiral = build_spiral(n, df_spiral)
    list_diagonal_1 = list(np.diag(df_spiral))
    
    list_columns_reflected = list(df_spiral.columns)[::-1]
    df_spiral_reflected = df_spiral[list_columns_reflected]
    list_diagonal_2 = list(np.diag(df_spiral_reflected))
    
    list_diagonals = list(set(list_diagonal_1+list_diagonal_2))

    
    list_prime_diagonals = [x for x in list_diagonals if is_prime(x)]
    
    if len(list_prime_diagonals)/float(len(list_diagonals)) < 0.1:
        print(n, df_spiral.shape)
        found = True

In [208]:
build_spiral(10, 0)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,325,324,323,322,321,320,319,318,317,316,315,314,313,312,311,310,309,308,307
1,326,257,256,255,254,253,252,251,250,249,248,247,246,245,244,243,242,241,306
2,327,258,197,196,195,194,193,192,191,190,189,188,187,186,185,184,183,240,305
3,328,259,198,145,144,143,142,141,140,139,138,137,136,135,134,133,182,239,304
4,329,260,199,146,101,100,99,98,97,96,95,94,93,92,91,132,181,238,303
5,330,261,200,147,102,65,64,63,62,61,60,59,58,57,90,131,180,237,302
6,331,262,201,148,103,66,37,36,35,34,33,32,31,56,89,130,179,236,301
7,332,263,202,149,104,67,38,17,16,15,14,13,30,55,88,129,178,235,300
8,333,264,203,150,105,68,39,18,5,4,3,12,29,54,87,128,177,234,299
9,334,265,204,151,106,69,40,19,6,1,2,11,28,53,86,127,176,233,298


_____

# Despite all this effort writing the code, it's too slow

## Redoing it

_____

### We notice the diagonal from the centre to the bottom right goes [1,9,25,49,81,...] which is equal to $[1^{2},3^{2},5^{2},,7^{2},,9^{2},...]$

### The diagonal from the centre to the bottom left is [1,7,21,43,73,...] which is equal to $[1^{2},3^{2},5^{2},,7^{2},,9^{2},...]$ - [0, 2, 4, 6, 8, ...]

### The diagonal from the centre to the top left is [1,5,17,37,65,...] which is equal to $[1^{2},3^{2},5^{2},,7^{2},,9^{2},...]$ - [0, 4, 8, 12, ...]

### The diagonal from the centre to the top right is [1,3,13,31,57,...] which is equal to $[1^{2},3^{2},5^{2},,7^{2},,9^{2},...]$ - [0, 6, 12, 18, ...]

## So, for a spiral of $n$ layers, we can build the diagonals as:

## `list_diagonal_1 = (1+2*np.arange(n))**2`

## `list_diagonal_2 = list_diagonal_1 - 2*np.arange(n)`

## `list_diagonal_3 = list_diagonal_1 - 3*np.arange(n)`

## `list_diagonal_4 = list_diagonal_1 - 6*np.arange(n)`

In [198]:
n = 0

found = False

list_diagonals = [1]
list_primes = []

while not found:
    n+=1
    new_val = (1+2*n)**2
    list_diagonals.append(new_val)
    if is_prime(new_val):
        list_primes.append(new_val)
    
    new_val -= 2*n
    list_diagonals.append(new_val)
    if is_prime(new_val):
        list_primes.append(new_val)
    
    new_val -= 2*n
    list_diagonals.append(new_val)
    if is_prime(new_val):
        list_primes.append(new_val)
    
    new_val -= 2*n
    list_diagonals.append(new_val)
    if is_prime(new_val):
        list_primes.append(new_val)

    if len(list_primes)/float(len(list_diagonals))<0.1:
        print(2*n+1)
        found = True

26241


# Answer: 26241